<h1>Modelos Lineales Mixtos</h1>
<h2> Ejemplo guía #1 </h2>

<img src="figuras/7_fig_arboles.png" width="500">

$$ y_i = \alpha_j + \beta_k + \epsilon_i $$
si la observacion $i$ es para el nivel $j$ de CO2 del arbol $k$ <br>

Datos

In [1]:
library(gamair)
data(stomata)

stomata

,area,CO2,tree
,<dbl>,<fct>,<fct>
1,1.6055739,1,1
2,1.6300711,1,1
3,1.5391189,1,1
4,1.7187315,1,1
5,1.3896163,1,2
6,1.5858805,1,2
7,1.4697276,1,2
8,1.9493473,1,2
9,1.5397020,1,3


Primero compare modelos con y sin el factor de árbol $\beta_k$:

$$ y_i = \alpha_j + \beta_k + \epsilon_i $$

In [2]:
m0 <- lm(area ~ CO2, data = stomata)
m1 <- lm(area ~ CO2 + tree, data = stomata)

anova(m0, m1)

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,22,2.1348274,NA,NA,NA,NA
2,18,0.8603993,4,1.274428,6.665424,0.001787974


Claramente, existe evidencia sólida de diferencias entre árboles, lo que significa que con este modelo no podemos decir si el CO2 tuvo un efecto o no.<br>Para volver a enfatizar este punto, esto es lo que sucede si intentamos probar el efecto del CO2:

Para volver a enfatizar este punto, esto es lo que sucede si intentamos probar el efecto del CO2: 

In [3]:
m2 <- lm(area ~ tree, stomata)

anova(m2, m1)

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,18,0.8603993,NA,NA,NA,NA
2,18,0.8603993,0,-1.110223e-16,NA,NA


La confusión del CO2 y los tres factores significa que los modelos que se comparan aquí
son en realidad el mismo modelo; "compararlos" no nos dice
nada sobre el efecto del CO2

Tratar los árboles individuales, no como individuos completamente únicos sino como unamuestra aleatoria de la población de árboles objetivo, nos permitirá estimar el efecto del CO2 y generalizar más allá de los 6 árboles del experimento

<h3>El enfoque correcto: un modelo de efectos mixtos</h3>

 el efecto del CO2 se modelará como un efecto fijo, pero el efecto del árbol se
modelará como un efecto aleatorio 

$$ y_i = \alpha_j + b_k + \epsilon_i $$

Si la observación $i$ es para el nivel $j$ del CO2, y el árbol $k$, donde ahora, 
$$b_k \sim N(0,\sigma_b^2)$$
$$ \epsilon_i \sim N(0,\sigma^2) $$
y las $b_k$ y $\epsilon_i$ son v.a. mutuamente independientes

... <br>
Es fácil ver que el modelo para el área estomática promedio por árbol debe ser
* $\overline{y_k} = \alpha_j + e_k$
* si el arbol $k$ está en el nivel $j$ de CO2, y donde $e_k \sim N(0, \frac{\sigma_b^2 + \sigma^2}{4})$
* hint: $Var(1/4(\epsilon_1+\epsilon_2+\epsilon_3+\epsilon_4))$


<h3>Usando R</h3>
Ahora es sencillo probar el efecto del CO2 en R, primero agregue los datos de cada árbol:

* utilizando la función $\texttt{aggregate()}$ para calcular la media de todas las columnas numéricas en el dataframe $\texttt{stomata}$, agrupadas por la columna $\texttt{tree}$. El resultado es un nuevo dataframe $\texttt{st}$ que contiene las medias de las columnas numéricas para cada valor único en la columna tree.

* <b><i>st$CO2 <- as.factor(st$CO2)</i></b>: Esta línea está convirtiendo la columna CO2 del dataframe st en un factor. Esto puede ser útil si CO2 se va a utilizar en un análisis que requiere que sea una variable categórica, como un análisis de varianza (ANOVA) o un modelo lineal generalizado (GLM).

In [4]:
st <- aggregate(data.matrix(stomata), by=list(tree=stomata$tree), mean)

st$CO2 <- as.factor(st$CO2)

st

tree,area,CO2,tree
<fct>,<dbl>,<fct>,<dbl>
1,1.623374,1,1
2,1.598643,1,2
3,1.162961,1,3
4,2.789238,2,4
5,2.903544,2,5
6,2.329761,2,6


y luego ajustar el modelo implícito en la agregación:

In [6]:
m3 <- lm(area ~ CO2, st)

anova(m3)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
CO2,1,2.205314,2.20531395,27.68695,0.006246682
Residuals,4,0.318607,0.07965176,NA,NA


Aquí hay pruebas sólidas de un efecto del CO2, y ahora procederíamos a examinar la estimación de este efecto fijo

In [7]:
summary(m3)


Call:
lm(formula = area ~ CO2, data = st)

Residuals:
      1       2       3       4       5       6 
 0.1617  0.1370 -0.2987  0.1151  0.2294 -0.3444 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1.4617     0.1629   8.970 0.000855 ***
CO22          1.2125     0.2304   5.262 0.006247 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2822 on 4 degrees of freedom
Multiple R-squared:  0.8738,	Adjusted R-squared:  0.8422 
F-statistic: 27.69 on 1 and 4 DF,  p-value: 0.006247


<b>Generalmente con un modelo mixto las varianzas de los efectosaleatorios son de más interés que los efectos mismos, por lo que en este ejemplo se debe estimar</b> $\sigma_b^2$

sea $RSS_i$ la suma residual de cuadrados del modelo $i$. <br>
De la teoría de modelos lineales tenemos que: <br>
* $\hat{\sigma^2} = RSS_2/18$ (modelos sin promediar) <br>
* $\hat{\sigma_b^2 + \sigma^2}/4 = RSS_3/4$ (modelos promediando) <br>

Ambos estimadores son insesgados <br>
Por tanto, un estimador insesgado para $\sigma_b^2$ es: <br>
$$\hat{\sigma_b^2} = \frac{\hat{\sigma_b^2 + \sigma^2}}{4} - 
    \frac{\hat{\sigma^2}}{4} = RSS_3/4 - RSS_2/72$$

In [8]:
summary(m3)$sigma^2 - summary(m2)$sigma^2 / 4

[1] 0.06770177